In [1]:
#Dependencies
import pandas as pd
import requests
import json

from keys import key
from API_Keys import gkey

In [11]:
#Read in city list (1000 most populous cities in US)
cities_df = pd.read_csv("Resources/US_Cities.csv")
cities_df.head()

cities_df

,City,State
0,New York,New York
1,Los Angeles,California
2,Chicago,Illinois
3,Houston,Texas
4,Philadelphia,Pennsylvania
...,...,...
995,Weslaco,Texas
996,Keizer,Oregon
997,Spanish Fork,Utah
998,Beloit,Wisconsin


In [12]:
#Get latitude and longitude for each city from Google Geocode API
cities_df["Latitude"] = ""
cities_df["Longitude"] = ""

params = {"key": gkey}

for index, row in cities_df.iterrows():
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    city1 = row["City"]
    state1 = row["State"]
    
    params['address'] = f"{city1},{state1}"
    
    cities_lat_lon = requests.get(url, params=params)
    
    cities_lat_lon = cities_lat_lon.json()
    
    cities_df.loc[index, "Latitude"] = cities_lat_lon["results"][0]["geometry"]["location"]["lat"]
    cities_df.loc[index, "Longitude"] = cities_lat_lon["results"][0]["geometry"]["location"]["lng"]
    
cities_df.head()

,City,State,Latitude,Longitude
0,New York,New York,40.7128,-74.006
1,Los Angeles,California,34.0522,-118.244
2,Chicago,Illinois,41.8781,-87.6298
3,Houston,Texas,29.7604,-95.3698
4,Philadelphia,Pennsylvania,39.9526,-75.1652


In [14]:
output_data_file = "resources/P1_cities_LL.csv"
cities_df.to_csv(output_data_file, index=False, header=True)

In [16]:
#Add columns to data set to hold weather data for each month
LOM = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

for mon in LOM:
    cities_df[f"{mon}_temp"] = ""
    cities_df[f"{mon}_precip"] = ""
    cities_df[f"{mon}_humid"] = ""
    cities_df[f"{mon}_cloud"] = ""
    
cities_df.head(10)

,City,State,Latitude,Longitude,Jan_temp,Jan_precip,Jan_humid,Jan_cloud,Feb_temp,Feb_precip,...,Oct_humid,Oct_cloud,Nov_temp,Nov_precip,Nov_humid,Nov_cloud,Dec_temp,Dec_precip,Dec_humid,Dec_cloud
0,New York,New York,40.7128,-74.006,,,,,,,...,,,,,,,,,,
1,Los Angeles,California,34.0522,-118.244,,,,,,,...,,,,,,,,,,
2,Chicago,Illinois,41.8781,-87.6298,,,,,,,...,,,,,,,,,,
3,Houston,Texas,29.7604,-95.3698,,,,,,,...,,,,,,,,,,
4,Philadelphia,Pennsylvania,39.9526,-75.1652,,,,,,,...,,,,,,,,,,
5,Phoenix,Arizona,33.4484,-112.074,,,,,,,...,,,,,,,,,,
6,San Antonio,Texas,29.4241,-98.4936,,,,,,,...,,,,,,,,,,
7,San Diego,California,32.7157,-117.161,,,,,,,...,,,,,,,,,,
8,Dallas,Texas,32.7767,-96.797,,,,,,,...,,,,,,,,,,
9,San Jose,California,37.3382,-121.886,,,,,,,...,,,,,,,,,,


In [17]:
#First API Call to Weather API using entire data set
#Discovered there were size / key limitations per API call
base_url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/historysummary?aggregateHours=24&minYear=2017&maxYear=2019&chronoUnit=months&breakBy=self&dailySummaries=false&contentType=json&unitGroup=us&locationMode=lookup&key='


for index, row in cities_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    query_url = base_url + key + "&locations=" + str(latitude) + "%2C%20" + str(longitude)
    
    response = requests.get(query_url).json()
    
    try:
        
        print(f"Extracting results for {latitude}, {longitude}:")
    
        for x in range(0, 12):
    
            month = response["locations"][f"{latitude}, {longitude}"]["values"][x]["period"]
    
            cities_df.loc[index, f"{month}_temp"] = response["locations"][f"{latitude}, {longitude}"]["values"][x]["temp"]
            cities_df.loc[index, f"{month}_precip"] = response["locations"][f"{latitude}, {longitude}"]["values"][x]["precip"]
            cities_df.loc[index, f"{month}_humid"] = response["locations"][f"{latitude}, {longitude}"]["values"][x]["humidity"]
            cities_df.loc[index, f"{month}_cloud"] = response["locations"][f"{latitude}, {longitude}"]["values"][x]["cloudcover"]
    
    except (KeyError, IndexError):
        print("Missing field/result...skipping.")
    
cities_df.head(10)
  

Extracting results for 40.7127753, -74.0059728:
Extracting results for 34.0522342, -118.2436849:
Extracting results for 41.8781136, -87.6297982:
Extracting results for 29.7604267, -95.3698028:
Extracting results for 39.9525839, -75.1652215:
Extracting results for 33.4483771, -112.0740373:
Extracting results for 29.4241219, -98.49362819999999:
Extracting results for 32.715738, -117.1610838:
Extracting results for 32.7766642, -96.79698789999999:
Extracting results for 37.3382082, -121.8863286:
Extracting results for 30.267153, -97.7430608:
Extracting results for 39.768403, -86.158068:
Extracting results for 30.3321838, -81.65565099999999:
Extracting results for 37.7749295, -122.4194155:
Extracting results for 39.9611755, -82.99879419999999:
Extracting results for 35.2270869, -80.8431267:
Extracting results for 32.7554883, -97.3307658:
Extracting results for 42.331427, -83.0457538:
Extracting results for 31.7618778, -106.4850217:
Extracting results for 35.1495343, -90.0489801:
Extracting 

Extracting results for 44.9428975, -123.0350963:
Missing field/result...skipping.
Extracting results for 34.6867846, -118.1541632:
Missing field/result...skipping.
Extracting results for 33.8752935, -117.5664384:
Missing field/result...skipping.
Extracting results for 44.0520691, -123.0867536:
Missing field/result...skipping.
Extracting results for 34.5794343, -118.1164613:
Missing field/result...skipping.
Extracting results for 36.6777372, -121.6555013:
Missing field/result...skipping.
Extracting results for 42.1014831, -72.589811:
Missing field/result...skipping.
Extracting results for 29.6910625, -95.2091006:
Missing field/result...skipping.
Extracting results for 40.5852602, -105.084423:
Missing field/result...skipping.
Extracting results for 37.6687665, -122.0809964:
Missing field/result...skipping.
Extracting results for 34.055103, -117.7499909:
Missing field/result...skipping.
Extracting results for 35.79154, -78.7811169:
Missing field/result...skipping.
Extracting results for 4

Extracting results for 27.9658533, -82.8001026:
Missing field/result...skipping.
Extracting results for 41.5581525, -73.0514965:
Missing field/result...skipping.
Extracting results for 45.5098502, -122.4347608:
Missing field/result...skipping.
Extracting results for 38.2491956, -122.0405151:
Missing field/result...skipping.
Extracting results for 45.7832856, -108.5006904:
Missing field/result...skipping.
Extracting results for 42.6334247, -71.31617179999999:
Missing field/result...skipping.
Extracting results for 34.2804923, -119.2945199:
Missing field/result...skipping.
Extracting results for 38.2544472, -104.6091409:
Missing field/result...skipping.
Extracting results for 35.9556923, -80.0053176:
Missing field/result...skipping.
Extracting results for 34.0686208, -117.9389526:
Missing field/result...skipping.
Extracting results for 37.9357576, -122.3477486:
Missing field/result...skipping.
Extracting results for 33.5539143, -117.2139232:
Missing field/result...skipping.
Extracting re

Extracting results for 37.7257663, -122.1568554:
Missing field/result...skipping.
Extracting results for 40.3356483, -75.9268747:
Missing field/result...skipping.
Extracting results for 41.11774399999999, -73.4081575:
Missing field/result...skipping.
Extracting results for 35.3859242, -94.39854749999999:
Missing field/result...skipping.
Extracting results for 33.6188829, -117.9298493:
Missing field/result...skipping.
Extracting results for 35.5950581, -82.5514869:
Missing field/result...skipping.
Extracting results for 42.7653662, -71.46756599999999:
Missing field/result...skipping.
Extracting results for 35.6528323, -97.47809540000002:
Missing field/result...skipping.
Extracting results for 33.9791793, -118.032844:
Missing field/result...skipping.
Extracting results for 43.5788175, -116.55978:
Missing field/result...skipping.
Extracting results for 44.840798, -93.2982799:
Missing field/result...skipping.
Extracting results for 28.9005446, -81.26367379999999:
Missing field/result...ski

Extracting results for 42.9133602, -85.7053085:
Missing field/result...skipping.
Extracting results for 30.2265949, -93.2173758:
Missing field/result...skipping.
Extracting results for 45.0105194, -93.4555093:
Missing field/result...skipping.
Extracting results for 41.69864159999999, -88.0683955:
Missing field/result...skipping.
Extracting results for 26.1947962, -98.1836216:
Missing field/result...skipping.
Extracting results for 44.2619309, -88.41538469999999:
Missing field/result...skipping.
Extracting results for 35.262082, -81.18730049999999:
Missing field/result...skipping.
Extracting results for 38.6779591, -121.1760583:
Missing field/result...skipping.
Extracting results for 42.4733688, -83.2218731:
Missing field/result...skipping.
Extracting results for 42.65836609999999, -83.1499322:
Missing field/result...skipping.
Extracting results for 41.6612104, -72.7795419:
Missing field/result...skipping.
Extracting results for 33.4353394, -112.3576567:
Missing field/result...skipping.

...

Extracting results for 28.8028612, -81.269453:
Missing field/result...skipping.
Extracting results for 39.68950359999999, -84.1688274:
Missing field/result...skipping.
Extracting results for 40.4862157, -74.4518188:
Missing field/result...skipping.
Extracting results for 34.6059253, -86.9833417:
Missing field/result...skipping.
Extracting results for 42.1487043, -72.6078672:
Missing field/result...skipping.
Extracting results for 40.1053196, -85.6802541:
Missing field/result...skipping.
Extracting results for 26.2445263, -80.206436:
Missing field/result...skipping.
Extracting results for 42.2180724, -70.94103559999999:
Missing field/result...skipping.
Extracting results for 40.7062128, -73.6187397:
Missing field/result...skipping.
Extracting results for 44.5645659, -123.2620435:
Missing field/result...skipping.
Extracting results for 33.952463, -117.5848025:
Missing field/result...skipping.
Extracting results for 36.06523, -119.0167679:
Missing field/result...skipping.
Extracting resul

Extracting results for 32.5093109, -92.1193012:
Missing field/result...skipping.
Extracting results for 33.8583483, -118.0647871:
Missing field/result...skipping.
Extracting results for 41.8089191, -88.01117459999999:
Missing field/result...skipping.
Extracting results for 25.7491968, -80.2635411:
Missing field/result...skipping.
Extracting results for 35.7212689, -77.9155395:
Missing field/result...skipping.
Extracting results for 43.0962143, -79.0377388:
Missing field/result...skipping.
Extracting results for 32.9628232, -117.0358646:
Missing field/result...skipping.
Extracting results for 44.8896866, -93.3499489:
Missing field/result...skipping.
Extracting results for 41.1339449, -81.48455849999999:
Missing field/result...skipping.
Extracting results for 33.640171, -117.602832:
Missing field/result...skipping.
Extracting results for 40.2731911, -76.8867008:
Missing field/result...skipping.
Extracting results for 38.4192496, -82.44515400000002:
Missing field/result...skipping.
Extrac

Extracting results for 25.5808323, -80.34685929999999:
Missing field/result...skipping.
Extracting results for 26.1723065, -80.1319893:
Missing field/result...skipping.
Extracting results for 25.9331488, -80.1625463:
Missing field/result...skipping.
Extracting results for 40.6576022, -73.58318349999999:
Missing field/result...skipping.
Extracting results for 41.5067003, -90.51513419999999:
Missing field/result...skipping.
Extracting results for 33.6803003, -116.173894:
Missing field/result...skipping.
Extracting results for 27.4467056, -80.3256056:
Missing field/result...skipping.
Extracting results for 35.9828412, -86.5186045:
Missing field/result...skipping.
Extracting results for 40.8893895, -111.880771:
Missing field/result...skipping.
Extracting results for 43.7730448, -88.4470508:
Missing field/result...skipping.
Extracting results for 42.40843, -71.0536625:
Missing field/result...skipping.
Extracting results for 36.5859718, -79.39502279999999:
Missing field/result...skipping.
Ex

Extracting results for 28.5691677, -81.5439619:
Missing field/result...skipping.
Extracting results for 44.7973962, -93.5272861:
Missing field/result...skipping.
Extracting results for 42.4792618, -71.1522765:
Missing field/result...skipping.
Extracting results for 47.5650067, -122.6269768:
Missing field/result...skipping.
Extracting results for 41.5094771, -90.5787476:
Missing field/result...skipping.
Extracting results for 35.7478769, -95.3696909:
Missing field/result...skipping.
Extracting results for 37.3058839, -89.51814759999999:
Missing field/result...skipping.
Extracting results for 38.9784453, -76.4921829:
Missing field/result...skipping.
Extracting results for 26.6276276, -80.1353896:
Missing field/result...skipping.
Extracting results for 29.2858129, -81.0558894:
Missing field/result...skipping.
Extracting results for 25.9812024, -80.14837899999999:
Missing field/result...skipping.
Extracting results for 33.8028875, -117.9931203:
Missing field/result...skipping.
Extracting r

,City,State,Latitude,Longitude,Jan_temp,Jan_precip,Jan_humid,Jan_cloud,Feb_temp,Feb_precip,...,Oct_humid,Oct_cloud,Nov_temp,Nov_precip,Nov_humid,Nov_cloud,Dec_temp,Dec_precip,Dec_humid,Dec_cloud
0,New York,New York,40.7128,-74.006,35,3.6,66.53,4,38.8,2.9,...,72.01,3.9,46.2,2.8,66.21,7.5,39.6,3.7,68.37,3.1
1,Los Angeles,California,34.0522,-118.244,58.1,5.1,64.29,33.3,56.8,3,...,56.49,20.1,64.1,1.2,59.53,22.7,58.8,2.3,54.64,16.9
2,Chicago,Illinois,41.8781,-87.6298,26.5,1.9,72.65,29.5,28.6,2.8,...,69.6,26.2,36.3,2.1,70.47,31.1,34.3,1.5,71.57,26.6
3,Houston,Texas,29.7604,-95.3698,55.2,2.1,66.22,32.4,62.5,2.7,...,69.82,31.3,62.8,1.5,70.21,38.9,58.2,3.9,66.35,42.2
4,Philadelphia,Pennsylvania,39.9526,-75.1652,34.7,3.3,63.65,54,39.9,3.5,...,70.58,49.2,44.5,3.3,64.19,47.4,38,4,65.68,52.8
5,Phoenix,Arizona,33.4484,-112.074,57,0.7,48.39,53.3,58.3,1.2,...,32.03,29.4,66.4,0.6,36.39,47.8,56.3,0.4,45.26,52
6,San Antonio,Texas,29.4241,-98.4936,53.5,1.1,61.76,47.1,60.3,2.4,...,66.97,48.8,60.5,0.9,69.1,50.1,53.9,2.2,66.34,45.5
7,San Diego,California,32.7157,-117.161,58.4,2.5,65.84,58.9,57.8,2.7,...,64.36,42.7,63.8,1.1,65.64,54.6,59,2.2,62.86,52.9
8,Dallas,Texas,32.7767,-96.797,47.6,1.6,59.93,48.9,53.6,4.4,...,64.7,45.3,56.1,1.7,64.41,49.8,49.9,3.4,63.51,52.1
9,San Jose,California,37.3382,-121.886,52.2,3.9,76.44,65.8,52.5,3.8,...,56.74,28.6,57,1.4,66.28,51.3,52,1.2,70.06,53.1


In [18]:
output_data_file = "resources/P1_cities_WD.csv"
cities_df.to_csv(output_data_file, index=False, header=True)